In [ ]:
import subprocess
import shlex
import datetime
from dateutil.tz import tzlocal
import pandas as pd
import glob
import time
from time import sleep 
def run_command_async(command):
    if (isinstance(command, list) == False):
        commands = [command]
    else:
        commands=command
    processes = [None] * len(commands)
    for i in range(len(commands)):
        processes[i] = subprocess.Popen(shlex.split(commands[i]), stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    return processes

def wait_for_command_async(processes):
    while processes:
        for i, process in enumerate(processes):
            output = process.stdout.readline().decode()
            if output == '' and process.poll() is not None:
                processes.remove(process)
                break
            if output:
                now=datetime.datetime.now(tzlocal())
                strnow = now.strftime("%Y-%m-%d %H:%M:%S")
                print ("Log {0} - {1} : ".format(i,strnow) + output.strip())
    
    rc = process.poll()
    return rc    
def run_command(command, shell=False):
    print(command)
    if (isinstance(command, list) == False):
        commands = [command]
    else:
        commands=command
    processes = [None] * len(commands)
    for i in range(len(commands)):
        print (commands[i])
        processes[i] = subprocess.Popen(shlex.split(commands[i]), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=shell)
    while processes:
        for i, process in enumerate(processes):
            output = process.stdout.readline().decode()
            if output == '' and process.poll() is not None:
                processes.remove(process)
                break
            if output:
                now=datetime.datetime.now(tzlocal())
                strnow = now.strftime("%Y-%m-%d %H:%M:%S")
                print ("Log {0} - {1} : ".format(i,strnow) + output.strip())
    
    rc = process.poll()
    return rc
import os

class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)
def get_strnow():
    now=datetime.datetime.now(tzlocal())
    return now.strftime("%Y%m%d%H%M%S")


In [ ]:
def wait_for_http_status(url, code, duration):
    import requests
    import time
    print
    req_code = None
    start_time = time.time()
    end_time= time.time()
    while req_code != code and (end_time - start_time) < duration :
        try:
            time.sleep(1)
            r = requests.head(url)
            req_code = r.status_code
            print("\rWaiting status {} for {} seconds            ".format(code, end_time - start_time), end="\r", flush=True)
            return 0
        except requests.ConnectionError:
            print("failed to connect")
            return 1
        end_time = time.time()    
        
def install_recsys(params):
    
    run_command("helm init --client-only")
    #launch commands manually
    #with cd("PrivateRecSys/charts/private-recsys"):
    #   run_command("helm repo add main https://kubernetes-charts.storage.googleapis.com/")
    #    run_command("helm repo add elastic https://helm.elastic.co")
    #    run_command("helm dep update") # can't work https://github.com/helm/helm/issues/2998 : it should be done on the node
        
    with cd("../charts/private-recsys"):
        run_command("kubectl apply -f ..")
        param_str = ""
        for k in params.keys():
            param_str += "--set {}={} ".format(str(k).replace("!","."), str(params[k]).replace("!","."))
        command = "helm install --name recsys --namespace default . {} --wait --timeout {} ".format(param_str, params["timeout"])
        result = run_command(command)
        return result    
    
def uninstall_recsys():
    with cd("../charts/private-recsys"):
        result = run_command("helm delete --purge recsys")
        if result == 0:
            sleep(120)
    return result

def install_stub(params):
    
    run_command("helm init --client-only")
    #launch commands manually
    #with cd("PrivateRecSys/charts/private-recsys"):
    #   run_command("helm repo add main https://kubernetes-charts.storage.googleapis.com/")
    #    run_command("helm repo add elastic https://helm.elastic.co")
    #    run_command("helm dep update") # can't work https://github.com/helm/helm/issues/2998 : it should be done on the node
        
    with cd("../charts/stub-recsys"):
        param_str = ""
        for k in params.keys():
            param_str += "--set {}={} ".format(str(k).replace("!","."), str(params[k]).replace("!","."))
        command = "helm install --name stub --namespace default . {} --wait --timeout {} ".format(param_str, params["timeout"])
        result = run_command(command)
        return result    

def uninstall_stub():
    with cd("../charts/stub-recsys"):
        result = run_command("helm delete --purge stub")
        if result == 0:
            sleep(30)
    return result

def launch_injector(params):
    with cd("../charts/injector"):
        param_str = ""
        for k in params.keys():
            param_str += "--set {}={} ".format(str(k).replace("!","."), str(params[k]).replace("!","."))
        command = "helm install --name injector --namespace default . {} --wait --timeout {}".format(param_str, params["timeout"])
        result = run_command(command)
        if result == 0:
            result = run_command("kubectl wait --namespace default --for=condition=complete --timeout={}s job/injector-injector".format(params["timeout"]+300))
        return result
    
def uninstall_injector():
    with cd("../charts/injector"):
        result = run_command("helm delete --purge injector")
        if result == 0:
            sleep(30)
    return result


In [ ]:
from pymongo import MongoClient
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns

client = MongoClient("manager-mongo-manager")

db = client["private-recsys"]
collection_experiment = db["experiments"]



In [ ]:
def is_harness_result_ok(collection):
    row = collection.find_one({"http_status":201})
    if row is not None:
        #print(row["content"]["result"])
        if len(row["content"]["result"]) > 0:
            return True
    else:
        print("Empty db")
    return False

def run_test(params):
    # uninstall previous launches
    print(uninstall_stub())
    print(uninstall_injector())
    print(uninstall_recsys())

    # init naming for experiment
    strnow = get_strnow()
    params_recsys = params["params_recsys"]
    params_injector = params["params_injector"]
    
    params["name"] = params["name"] + strnow
    params_injector["name"] = params_injector["name"] + strnow
    params_recsys["name"] = params_recsys["name"] + strnow
    
    mongo_id = collection_experiment.insert_one(params).inserted_id
    collection_experiment.update_one({'_id':mongo_id}, {"$set": params}, upsert=False)

    # init step recsys
    params_recsys["start_time"] = time.time()
    params["status"] = "recsys"
    collection_experiment.update_one({'_id':mongo_id}, {"$set": params}, upsert=False)
    
    cpt = 0
    err = None
    while (err is None or err != 0) and cpt < 5:
        
        if params["stub"] != "1":
            err=install_recsys(params_recsys)
        else:
            err=install_stub(params_recsys)
        print("err={}".format(err))
        if err != 0:
            if params["stub"] != "1":
                uninstall_recsys()
            else:
                uninstall_stub()
        cpt += 1

    params_recsys["end_time"] = time.time()
    params["status"] = "recsys"
    collection_experiment.update_one({'_id':mongo_id}, {"$set": params}, upsert=False)

    if err != 0:
        return params["status"]
    
    params_injector["start_time"] = time.time()
    params["status"] = "injector"
    collection_experiment.update_one({'_id':mongo_id}, {"$set": params}, upsert=False)

    time.sleep(120)
    cpt = 0

    err = launch_injector(params_injector)
    print("Injector deployment err : {}".format(err))
    uninstall_injector()
    if err != 0:
        return params["status"]
    
    params_injector["end_time"] = time.time()
    params["status"] = "injector"
    collection_experiment.update_one({'_id':mongo_id}, {"$set": params}, upsert=False)

    params_injector["error"] = err
    params_injector["end_time"] = time.time()
    #now=datetime.datetime.now(tzlocal())
    #strnow = now.strftime("%Y%m%d%H%M%S")


    if params["stub"] != "1":
        uninstall_recsys()
    else:
        uninstall_stub()
    

    coll_name = "private-recsys.{}.locust_request_result".format(params_injector["name"]) 
    coll = db[coll_name]
    if is_harness_result_ok(coll) == True:
        params["status"] = "finished"
    else:
        params["status"] = "harness_error"
    print("{} - {}".format(coll_name, is_harness_result_ok(coll)))
        
    params["end_time"] = time.time()
    collection_experiment.update_one({'_id':mongo_id}, {"$set": params}, upsert=False)
    return params["status"]


In [ ]:
import itertools
import collections
def unpack_dict(target):
    ar = collections.OrderedDict()
    for k in target.keys():
        if isinstance(target[k], list):
            ar[k] = target[k]
    print(ar.keys())
    combinations = [dict(zip(ar.keys(), i)) for i in itertools.product(*ar.values())]
    result = []
    for c in combinations:
        print(c)
        d = target.copy()
        d.update(c)
        result.append(d)
    return result, ar

def rename_db(old, new):
    import pymongo
    from pymongo import MongoClient
    client = MongoClient("manager-mongo-manager")
    client.admin.command('copydb',
                         fromdb=old,
                         todb=new)
    client.drop_database(old)
    
def loop_run_test(params_packed):
    list_params_recsys, arrays_params_recsys = unpack_dict(params_packed["params_recsys"])
    list_params_injector, arrays_params_injector = unpack_dict(params_packed["params_injector"])

    strnow = get_strnow()
    print("db:" + "-".join(list(arrays_params_recsys))+"_"+"-".join(list(arrays_params_injector))+"_"+strnow)

    print(list_params_injector)
    for params_recsys in list_params_recsys:
        for params_injector in list_params_injector:
            for i in range(5):
                params = params_packed.copy()
                params["params_recsys"] = params_recsys.copy()
                params["params_injector"] = params_injector.copy()
                result = run_test(params.copy())
                if result == "recsys" or result == "finished":
                    break
    #rename database 
    rename_db("private-recsys", "-".join(list(arrays_params_recsys))+"_"+"-".join(list(arrays_params_injector))+"_"+strnow)

In [ ]:
name = "tuning"
params_recsys = {
    "mongo!replicaSet!replicas!secondary":0,
    "elastic!replicas":2,
    "harness!replicaCount":1,
    "proxy-sgx!replicaCount":1,
    "proxy-sgx!proxyType1":"1",
    "proxy-sgx!proxyType2":"2",
    "proxy-sgx!enableEncryption":"1",
    "mongo!replicaSet!enabled": "false",
    "name":name,
    "timeout":450,
    "mongo!clusterDomain":"nuc.local"  # Louvain
#    "mongo!clusterDomain":"nuc.local" # Bordeaux
}
params_injector={
    "duration":60,
    "clients":[1,1,1,10,10,10,50,50,50,100,100,100],
    "replicaCount":1,
#    "targetHost":["http://recsys-proxy-sgx-1:8080"], # pprox
#    "targetHost":["http://stub-stub-recsys:8080"],   # stub
    "targetHost":["http://recsys-harness:9090"],      # harness  

    "timeout":1200,
    "asyncFluentd":"false",
    "timeoutRequest":800,
    "periodRequest":1000,
    "enableEncryption":"0",
    "name":name,
    "harness":"true" # false if stub
}
params = {
    "start_time":time.time(),
    "params_recsys":params_recsys,
    "params_injector":params_injector,
    "name:":name,
    "error_module":"",
    "error":0,
    "status":"created",
    "name":name,
    "stub":"0"
}

loop_run_test(params)

In [ ]:
# reset 

print(uninstall_stub())
print(uninstall_injector())
print(uninstall_recsys())